In [ ]:
import pandas as pd
import os,sys,codecs,csv,pickle,string
from pprint import pprint
import numpy as np
from stop_words import get_stop_words
import multiprocessing, datetime

import bz2
import pickle
import _pickle as cPickle


In [4]:
# create a list of the CSV files and the relevant year
#csv_files = [f for f in os.listdir() if f.endswith('.csv') ]
#csv_files = [(d,s) for d in range(4000) for s in csv_files if s.endswith('FY'+str(d)+'.csv') ]

# only abstract files
csv_files = [f for f in os.listdir() if (f.endswith('.csv') and f.find('_PRJABS_')>-1) ]
csv_files = [(d,s) for d in range(4000) for s in csv_files if s.endswith('FY'+str(d)+'.csv') ]
pprint(csv_files)


[(2004, 'RePORTER_PRJABS_C_FY2004.csv'),
 (2005, 'RePORTER_PRJABS_C_FY2005.csv'),
 (2006, 'RePORTER_PRJABS_C_FY2006.csv'),
 (2007, 'RePORTER_PRJABS_C_FY2007.csv'),
 (2008, 'RePORTER_PRJABS_C_FY2008.csv'),
 (2009, 'RePORTER_PRJABS_C_FY2009.csv'),
 (2010, 'RePORTER_PRJABS_C_FY2010.csv'),
 (2011, 'RePORTER_PRJABS_C_FY2011.csv'),
 (2012, 'RePORTER_PRJABS_C_FY2012.csv'),
 (2013, 'RePORTER_PRJABS_C_FY2013.csv'),
 (2014, 'RePORTER_PRJABS_C_FY2014.csv'),
 (2015, 'RePORTER_PRJABS_C_FY2015.csv'),
 (2016, 'RePORTER_PRJABS_C_FY2016.csv'),
 (2017, 'RePORTER_PRJABS_C_FY2017.csv'),
 (2018, 'RePORTER_PRJABS_C_FY2018.csv'),
 (2019, 'RePORTER_PRJABS_C_FY2019.csv')]


In [30]:
# save and load compressed pickle files
def decompress_pickle(filename):
    if os.path.isfile(filename):
        data = bz2.BZ2File(filename, 'rb')
    elif os.path.isfile( filename + '.pbz2' ):
        data = bz2.BZ2File(filename + '.pbz2'  , 'rb')
    else:
        print('cannot find ', filename)
        raise TypeError("Only integers are allowed")
    data = cPickle.load(data)
    return data

def compressed_pickle(filename, data):
    with bz2.BZ2File(filename + '.pbz2','w') as f: 
        cPickle.dump(data, f)


# grants_abs_list is a list of sets. each set contains all the words in one grant abstract
#   identify the sets in which the word appears
#   and return the #, avg, and total # of grants (# of sets)
def CountGrantsInOneYearWithWord( grants_abs_list , word):
    binary_vect = [word in words for words in grants_abs_list]
    return( sum(binary_vect), sum(binary_vect)/len(binary_vect)*100, len(binary_vect) )


# load an abstracts CSV file and return a set of unique, no-stop & no-punctuation words
#   also save the output as a pickle file
def ProcessesWordsInAbstractFile(abstract_csv_file,year):
    abstract_text = list()
    table = str.maketrans(dict.fromkeys(string.punctuation))
    with codecs.open( abstract_csv_file , 'r' , encoding='utf-8', errors='ignore') as csvfile:
        csvr = csv.reader(csvfile,delimiter=',')
        print('Starting with ', abstract_csv_file, ' saving to disk: ', datetime.datetime.now() )
        for app_id, app_txt in csvr:
            app_txt = [s.translate(table) for s in app_txt.lower().split(' ')]
            filtered_words = [word for word in app_txt  if \
                              ( (word not in get_stop_words('english')) and (len(word)>4) )]
            abstract_text.append(set(filtered_words))
    print('Finished with ', abstract_csv_file, ' saving to disk: ', datetime.datetime.now() )
    compressed_pickle(abstract_csv_file.replace('.csv',''), (year,abstract_text))
    return( abstract_text )

def TestProcess(x):
    #return(arg1)
    #print('Running TestProcess: ', arg1, arg2)
    #print('current_process: ', multiprocessing.current_process())
    #print('parent_process: ', multiprocessing.parent_process())
    return(x)




In [31]:
# in parallel, process the CSV files, and save the output in pickle format
if __name__ == '__main__':
    with multiprocessing.Pool(processes=4) as pool:              # start 4 worker processes
        print('Found ' , multiprocessing.cpu_count() , ' processors.')
        multiple_results = [pool.apply_async(TestProcess, (i)) for i in range(4)]
        print([res.get(timeout=1) for res in multiple_results])
        #print(pool.map(TestProcess,[csv_files[I][1],csv_files[I][0]]))
        #for I in range(len(csv_files)):
        #    pool.apply_async(TestProcess, [csv_files[I][1],csv_files[I][0]])
        #   result = pool.apply_async(ProcessesWordsInAbstractFile, [csv_files[I][1],csv_files[I][0]])
        #pool.close()
        #print('all proceses started')
        #pool.join()
        #print(' -- all finished --')
print('done')

Found  12  processors.


TimeoutError: 

In [ ]:
# load the pickle files
abstracts = list()
years = list()
for I in range(len(csv_files)):
    (year,abstract_text_list_of_sets) = decompress_pickle(csv_files[I][1].replace('.csv',''))
    abstracts.append(abstract_text_list_of_sets)
    years.append(year)
    print('Finished processing ', year)
abstracts = np.array(abstracts)
years = np.array(years)
print(years)

In [ ]:
print(len(abstracts[years==year][0]))
print(len(abstracts[years==year] ))
print(list(abstracts[1][1])[:25])  # first 100 words in one abstract in one year

In [ ]:
print( f'# of abstracts = {len(abstract_text_list)}')
print("0: ", abstract_text_list[0])
print("1: ", abstract_text_list[1])
s,m,l = CountGrantsInOneYearWithWord( abstract_text_list, 'yeast')
print(f'N={s}\t%={m}\ttotal N={l}')

#s,m,l = CountGrantsInOneYearWithWord( abstracts[years==2010] , 'yeast')
#print(s,m,l)

In [ ]:
unique_years = sorted(set(years))
keywords = ['yeast','cancer','zebrafish','elegans','crispr']
for keyword in keywords:
    for year in unique_years:
        s,m,l = CountGrantsInOneYearWithWord( abstracts[years==year][0] , keyword)
        print(year,keyword,m,sep='\t')

In [ ]:
# save the two lists

with open('abstracts.pickle', 'wb') as f:
    pickle.dump(abstracts, f, pickle.HIGHEST_PROTOCOL)
with open('years.pickle', 'wb') as f:
    pickle.dump(years, f, pickle.HIGHEST_PROTOCOL)

with open('abstracts.pickle', 'rb') as f:
    abstracts = pickle.load(f) 

with open('years.pickle', 'rb') as f:
    years = pickle.load(f)     

print(abstracts[2])